In [1]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm


# Text Preprocessing and NLP
import nltk

# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords

# Tokenizing sentences/words
from nltk.tokenize import word_tokenize

# Part-of-speech tagging
from nltk import pos_tag

# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer

In [2]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /Users/inflaton/code/engd/papers/DM-Fake-News-Detection


# Data Preparation (Loading CSV)

Load the processed_data `csv` file into pandas DataFrames
- `processed_data.csv` is loaded into `data` DataFrame (stemming has been performed to reduce processing time.)

In [3]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

/Users/inflaton/anaconda3/envs/fake-news/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

### Convolutional Neural network + Custom-trained word2vec word embeddings + 5-Fold Cross Validation + L2 Regularization + GridSearchCV


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Embedding,
    Conv1D,
    GlobalMaxPooling1D,
    Dense,
    Dropout,
    Input,
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from gensim.models import Word2Vec
from tensorflow.keras.regularizers import l2

# Set seeds for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


def train_word2vec_and_create_embeddings(
    train_texts, word_index, max_words, embedding_dim=100
):
    """Train Word2Vec on training data only and create embedding matrix"""
    # Train Word2Vec on training data only
    train_sentences = [text.split() for text in train_texts]
    word2vec_model = Word2Vec(
        sentences=train_sentences,
        vector_size=embedding_dim,
        window=5,
        min_count=2,
        workers=4,
    )

    # Create embedding matrix with correct dimensions
    vocab_size = min(max_words, len(word_index) + 1)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        if i < vocab_size:  # Only include words within max_words limit
            if word in word2vec_model.wv:
                embedding_matrix[i] = word2vec_model.wv[word]
            else:
                embedding_matrix[i] = np.random.normal(size=(embedding_dim,))

    return embedding_matrix


def create_model(
    max_sequence_length,
    vocab_size,
    embedding_dim,
    embedding_matrix,
    filters,
    dropout_rate,
):
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        trainable=True,
    )(input_layer)

    x = Conv1D(
        filters=filters, kernel_size=5, activation="relu", kernel_regularizer=l2(0.01)
    )(embedding_layer)

    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(dropout_rate)(x)
    output_layer = Dense(1, activation="sigmoid", kernel_regularizer=l2(0.01))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [5]:
def train_model(
    train_data,
    val_data,
    epochs=10,
    batch_size=64,
    filters=64,
    dropout_rate=0.3,
    grid_search=False,
):
    # Set seeds for reproducibility
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    # Constants
    max_words = 10000
    max_sequence_length = 300
    embedding_dim = 100

    print(f"\nTraining with paras: filters={filters}, dropout_rate={dropout_rate}")

    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Fit tokenizer on training data only
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences
    X_train = pad_sequences(
        tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length
    )
    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )

    # Get vocab size for this fold
    vocab_size = min(max_words, len(tokenizer.word_index) + 1)

    # Create embedding matrix using training data only
    embedding_matrix = train_word2vec_and_create_embeddings(
        train_texts, tokenizer.word_index, max_words, embedding_dim
    )

    # Create and train model
    model = create_model(
        max_sequence_length=max_sequence_length,
        vocab_size=vocab_size,
        embedding_dim=embedding_dim,
        embedding_matrix=embedding_matrix,
        filters=filters,
        dropout_rate=dropout_rate,
    )

    # Train model
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )

    # Evaluate using F1-score
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    # Store results
    result = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

    print("\nResult:")
    for key, value in result.items():
        print(f"\t{key}: {value}")

    return result if grid_search else model

In [15]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from tqdm import tqdm


def evaluate_model(model, train_data, val_data):
    print("Evaluating Model")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )
    y_pred = (model.predict(X_val) > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

In [6]:
def do_grid_search(data):
    # Define parameter grid
    param_grid = {"filters": [64, 128], "dropout_rate": [0.2, 0.3, 0.4, 0.5]}

    # Initialize variables to track results
    results = []
    best_score = 0
    best_params = None

    # Perform grid search with cross-validation
    for filters in param_grid["filters"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\nTesting filters={filters}, dropout_rate={dropout_rate}")

            # Initialize cross-validation
            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            # Perform k-fold cross-validation
            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(data["processed_full_content"], data["label"]), 1
            ):
                print(f"\nFold {fold}")

                # Split data
                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                result = train_model(
                    train_data,
                    val_data,
                    filters=filters,
                    dropout_rate=dropout_rate,
                    grid_search=True,
                )

                fold_score = result["f1_score"]
                fold_scores.append(fold_score)

                print(f"Fold {fold} F1-score: {fold_score:.4f}")

            # Calculate average score for this parameter combination
            avg_score = np.mean(fold_scores)
            print(f"Average F1-score: {avg_score:.4f}")

            # Store results
            results.append(
                {
                    "filters": filters,
                    "dropout_rate": dropout_rate,
                    "avg_f1_score": avg_score,
                    "fold_scores": fold_scores,
                }
            )

            # Update best parameters if necessary
            if avg_score > best_score:
                best_score = avg_score
                best_params = {"filters": filters, "dropout_rate": dropout_rate}

    # Print final results
    print("\nGrid Search Results:")
    for result in results:
        print(
            f"Filters: {result['filters']}, Dropout: {result['dropout_rate']}, "
            f"F1-score: {result['avg_f1_score']:.4f}"
        )

    print("\nBest Parameters:")
    print(f"Filters: {best_params['filters']}")
    print(f"Dropout Rate: {best_params['dropout_rate']}")
    print(f"Best F1-Score: {best_score:.4f}")

    results_df = pd.DataFrame(results)
    print("\nResults Summary:")
    print(results_df.sort_values("avg_f1_score", ascending=False))
    
    return best_params

In [9]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [10]:
%%time

best_params = do_grid_search(data)
best_params


Testing filters=64, dropout_rate=0.2

Fold 1

Training with paras: filters=64, dropout_rate=0.2
Epoch 1/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8846 - loss: 1.0193 - val_accuracy: 0.9643 - val_loss: 0.2598
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9622 - loss: 0.2432 - val_accuracy: 0.9685 - val_loss: 0.1840
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9686 - loss: 0.1813 - val_accuracy: 0.9731 - val_loss: 0.1578
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9733 - loss: 0.1611 - val_accuracy: 0.9740 - val_loss: 0.1518
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9757 - loss: 0.1513 - val_accuracy: 0.9744 - val_loss: 0.1466
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9791 - loss: 0.1408 - val_accuracy: 0.9760 - val_loss: 0.1395
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9825 - loss: 0.1318 - val_accuracy: 0.9761 - val_loss: 

{'filters': 128, 'dropout_rate': 0.3}

In [12]:
%%time

model = train_model(train_data, val_data, filters=best_params["filters"], dropout_rate=best_params["dropout_rate"])
model


Training with paras: filters=128, dropout_rate=0.3
Epoch 1/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.8860 - loss: 1.3371 - val_accuracy: 0.9626 - val_loss: 0.3061
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9616 - loss: 0.2748 - val_accuracy: 0.9696 - val_loss: 0.1930
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9685 - loss: 0.1882 - val_accuracy: 0.9701 - val_loss: 0.1678
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.9728 - loss: 0.1633 - val_accuracy: 0.9694 - val_loss: 0.1607
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9768 - loss: 0.1499 - val_accuracy: 0.9739 - val_loss: 0.1486
Epoch 6/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.9792 - loss: 0.1383 - val_accuracy: 0.9707 - val_loss: 0.1556
Epoch 7/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9808 - loss: 0.1308 - val_accuracy: 0.9729 - val_loss: 0.1466
Epoch 8/10
851/851 ━━━━━━━━━━━━━━━━━━━

<Functional name=functional_41, built=True>

In [13]:
model.save("results/CNN_model_original.keras")

In [14]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/CNN_model_original.keras")
model2.summary()

Model: "functional_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_41 (InputLayer)     │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_41 (Embedding)        │ (None, 300, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_41 (Conv1D)              │ (None, 296, 128)       │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_41         │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,217,349 (12.27 MB)

 Trainable params: 1,072,449 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,144,900 (8.18 MB)

In [17]:
%%time

evaluate_model(model, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.9706
Precision: 0.9820
Recall: 0.9501
F1: 0.9658
CPU times: user 11.3 s, sys: 695 ms, total: 12 s
Wall time: 8.28 s


In [18]:
%%time

evaluate_model(model2, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.9706
Precision: 0.9820
Recall: 0.9501
F1: 0.9658
CPU times: user 11.4 s, sys: 725 ms, total: 12.1 s
Wall time: 8.32 s


In [19]:
model3 = load_model("results/CNN_model.keras")
model3.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_5 (Embedding)         │ (None, 300, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 296, 64)        │        32,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,108,869 (11.86 MB)

 Trainable params: 1,036,289 (3.95 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,072,580 (7.91 MB)

In [20]:
%%time

evaluate_model(model3, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9762
Precision: 0.9750
Recall: 0.9705
F1: 0.9727
CPU times: user 9.65 s, sys: 654 ms, total: 10.3 s
Wall time: 8.14 s


In [27]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat([train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120982 entries, 0 to 120981
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   label                   120982 non-null  int64 
 1   full_content            120982 non-null  object
 2   processed_full_content  120982 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [25]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.8174
Precision: 0.9356
Recall: 0.6256
F1: 0.7498
CPU times: user 11.2 s, sys: 717 ms, total: 11.9 s
Wall time: 8.14 s


In [23]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Accuracy: 0.8174
Precision: 0.9356
Recall: 0.6256
F1: 0.7498
CPU times: user 11.3 s, sys: 710 ms, total: 12 s
Wall time: 8.25 s


In [24]:
%%time

evaluate_model(model3, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.8453
Precision: 0.9147
Recall: 0.7129
F1: 0.8013
CPU times: user 9.5 s, sys: 566 ms, total: 10.1 s
Wall time: 7.9 s


In [ ]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


Testing filters=64, dropout_rate=0.2

Fold 1

Training with paras: filters=64, dropout_rate=0.2


In [ ]:
model_rewritten = train_model(
    train_data_rewritten, val_data_rewritten, epochs=10, batch_size=64
)
model_rewritten.save("results/CNN_model_rewritten.keras")


Training with paras: filters=64, dropout_rate=0.3
Epoch 1/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.7988 - loss: 1.2049 - val_accuracy: 0.8702 - val_loss: 0.4386
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.8843 - loss: 0.3986 - val_accuracy: 0.8977 - val_loss: 0.3300
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.9002 - loss: 0.3233 - val_accuracy: 0.8940 - val_loss: 0.3235
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 17s 20ms/step - accuracy: 0.9083 - loss: 0.3001 - val_accuracy: 0.8954 - val_loss: 0.3151
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.9151 - loss: 0.2846 - val_accuracy: 0.9030 - val_loss: 0.2962
Epoch 6/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 18s 21ms/step - accuracy: 0.9208 - loss: 0.2696 - val_accuracy: 0.9056 - val_loss: 0.2956
Epoch 7/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.9303 - loss: 0.2550 - val_accuracy: 0.9028 - val_loss: 0.3002
Epoch 8/10
851/851 ━━━━━━━━━━━━━━━━━━━━

In [14]:
evaluate_model(model_rewritten, train_data_rewritten, val_data_rewritten)

Evaluating Model


190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9028
Precision: 0.8809
Recall: 0.8995
F1: 0.8901


In [15]:
evaluate_model(model_rewritten, train_data_rewritten, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.9321
Precision: 0.9729
Recall: 0.8689
F1: 0.9180


In [16]:
train_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(train_combined, val_combined, epochs=10, batch_size=64)
model_combined.save("results/CNN_model_combined.keras")


Training with paras: filters=64, dropout_rate=0.3
Epoch 1/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 36s 21ms/step - accuracy: 0.8616 - loss: 0.9830 - val_accuracy: 0.9224 - val_loss: 0.2842
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 333s 196ms/step - accuracy: 0.9256 - loss: 0.2716 - val_accuracy: 0.9314 - val_loss: 0.2425
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.9347 - loss: 0.2403 - val_accuracy: 0.9381 - val_loss: 0.2259
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 333s 196ms/step - accuracy: 0.9401 - loss: 0.2223 - val_accuracy: 0.9393 - val_loss: 0.2193
Epoch 5/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 336s 197ms/step - accuracy: 0.9444 - loss: 0.2115 - val_accuracy: 0.9407 - val_loss: 0.2158
Epoch 6/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.9485 - loss: 0.2034 - val_accuracy: 0.9412 - val_loss: 0.2118
Epoch 7/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 250s 147ms/step - accuracy: 0.9512 - loss: 0.1962 - val_accuracy: 0.9407 - val_loss: 0.2106
Epoch 8/10
1702/1

In [17]:
evaluate_model(model_combined, train_combined, val_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.9398
Precision: 0.9314
Recall: 0.9311
F1: 0.9312


In [18]:
evaluate_model(model_combined, train_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9704
Precision: 0.9706
Recall: 0.9615
F1: 0.9660


In [19]:
evaluate_model(model_combined, train_combined, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9093
Precision: 0.8929
Recall: 0.9006
F1: 0.8967
